In [ ]:
!wget https://github.com/maxpumperla/dl4j_coursera/releases/download/v0.4/dl4j-snapshot.jar

In [ ]:
!wget https://raw.githubusercontent.com/maxpumperla/dl4j_coursera/master/iris.txt

In [ ]:

import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

/gpfs/fs01/user/se2e-1a118c4f322670-980ba6aaa6c3/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:

# fix random seed for reproducibility
seed = 10
numpy.random.seed(seed)

In [ ]:
# load dataset
dataframe = pandas.read_csv("iris.txt", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

In [ ]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [ ]:
model = Sequential()
model.add(Dense(4, input_dim=4, activation='relu'))
model.add(Dense(3, activation='sigmoid'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X,dummy_y,epochs=20,batch_size=5)
model.save('iris_model.h5')

In [ ]:
#some learners constantly reported 502 errors in Watson Studio. 
#This is due to the limited resources in the free tier and the heavy resource consumption of Keras.
#This is a workaround to limit resource consumption

from keras import backend as K

K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))


In [1]:
!$SPARK_HOME/bin/spark-submit \
--class skymind.dsx.KerasImportCSVSparkRunner \
--files iris.txt,iris_model.h5 \
--master $MASTER \
dl4j-snapshot.jar \
-batchSizePerWorker 15 \
-indexLabel 4

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/src/spark21master/spark-2.1.2-bin-2.7.3/jars/slf4j-log4j12-1.7.16.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/src/wml-libs.v18/spark-2.0/jars/ml-event-client-scala-library-0.1.55-201709150512-allinone.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/src/wml-libs.v18/spark-2.0/jars/tika-app-2.0-1.14.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/src/dataconnector-dw-2.0/spark-2.0.0/Server/connectivity/thirdparty/slf4j-simple-1.6.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
18/03/01 06:27:41 INFO apache.spark.SparkContext: Running Spark version 2.1.2
18/03/01 06:27:41 WARN hadoop.util.NativeCodeLoader: Unable to

In [ ]:
!mv dl4j-snapshot.jar.1 dl4-snapshot.jar


In [3]:
!ls dl4j-snapshot.jar

dl4j-snapshot.jar
